In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets

In [3]:
ds = datasets.load_dataset("franlucc/py_all_mutations_v3_starcoderbase-1b", split="train")

In [4]:
ds

Dataset({
    features: ['mutated_program', 'mutations', 'key', 'prefix', 'suffix', 'middle', 'correct', 'model', 'fim_type', 'fim_program', 'hexsha', 'mutated_generated_text'],
    num_rows: 7320
})

In [13]:
print(ds[8]["mutated_program"])

import logging
import shutil
import os

from zerver.data_import.import_util import (
    build_attachment,
    create_converted_data_files,
)

from typing import Any, Dict, List, Optional

class AttachmentHandler:
    def __init__(__tmp3) -> None:
        __tmp3.info_dict = dict()  # type: Dict[str, Dict[str, Any]]

    def __tmp6(__tmp3,
                            realm_id: int,
                            __tmp0: <FILL>,
                            sender_id: int,
                            attachment: Dict[str, Any],
                            files_dir) -> Optional[str]:
        if not attachment:
            return None

        name = attachment['name']

        if 'path' not in attachment:
            logging.info('Skipping HipChat attachment with missing path data: ' + name)
            return None

        size = attachment['size']
        path = attachment['path']

        local_fn = os.path.join(files_dir, path)

        target_path = os.path.join(
            str(realm_i

In [16]:
ds = datasets.load_from_disk("/home/franlucc/projects_data/codetrace/results_v2/starcoderbase-1b/python/all_muts/ood_steering_results_ds")

In [17]:
ds

Dataset({
    features: ['steered_generation', 'correct_steer', 'fim_program', 'fim_type', 'hexsha'],
    num_rows: 631
})

In [20]:
print(ds[2]["fim_program"])

#!/usr/bin/env python3
# Copyright (C) 2021 Checkmk GmbH - License: GNU General Public License v2
# This file is part of Checkmk (https://checkmk.com). It is subject to the terms and
# conditions defined in the file COPYING, which is part of this source code package.

from collections.abc import Sequence

import pytest
from polyfactory.factories.pydantic_factory import ModelFactory
from pytest_mock import MockerFixture

from cmk.base.plugins.agent_based.agent_based_api.v1 import Attributes, TableRow
from cmk.base.plugins.agent_based.inventory_kube_node import inventory_kube_node

from cmk.plugins.lib.kube import (
    HealthZ,
    IpAddress,
    KubeletInfo,
    NodeAddress,
    NodeInfo,
    NodeName,
    Timestamp,
)

from .utils_inventory import sort_inventory_result


@pytest.mark.parametrize(
    "section_info, section_kubelet, expected_check_result",
    [
        pytest.param(
            NodeInfo(
                architecture="amd64",
                kernel_version="5.13.0-27-g